In [1]:
#import the necessary modules 
%matplotlib inline 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
#import scipy
import sklearn
import itertools as it
from itertools import cycle 
import os.path as op
import timeit 
import json
from matplotlib import animation
import matplotlib.font_manager as font_manager
from collections import namedtuple
#from functools import partial
#from pathlib import Path

In [2]:
# Set plotting style
plt.style.use('seaborn-white')

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
#import matplotlib.pyplot as plt

In [3]:
%matplotlib widget

In [4]:
import multiprocessing as m_proc
m_proc.cpu_count()

4

### Now use MD Analysis to calculate no. of frames a center PEG residues and terminal PEG residue is with 4 Angstroms of BSA (0.9 nm restrained system)

Import MDAnalysis

In [5]:
from prot_polymer_analysis import get_protresd_list, aa_frmcount, grptwocnt_aa, gtwo_trjcnt 
from prot_polymer_analysis import frac_cont, bavg_frac_cnt, prot_poly_cntmovie, AA_list_org

In [6]:
# Import MDAnalysis
import MDAnalysis as mda
import MDAnalysis.analysis.distances as maa_dist

### First table will be total fractional contacts and oligomer occupancy values for each Rg value 

#### Distance-based analysis 

Find residues that have at least one atom within a cutoff $d = 4.0$ Angstrom near water molecules in BSA/water simulation

Calculate the number of surface bsa residues from a 1 ns BSA/water simulation

In [7]:
#Units of Angstroms 
dmax = 4.0 

In [8]:
def middle_of_band(band_start, band_stop, plot_min=0, plot_max=60):
    half_way = (band_stop - band_start) / 2
    mid_band = band_start + half_way
    plot_fraction = (mid_band - plot_min) / (plot_max - plot_min)

    return plot_fraction

# 0.9 nm PEG unrestrained Rg 100 ns trajectory

Load the rg = 1.2 nm (3 PLGA N = 20 oligomer/BSA system) 

In [9]:
# Set up the MD Simulation, Make sure you do gmx trjconv -s topol.tpr -f confout.gro  -o new_conf.pdb 
# -dump 0 -n bsaplga_nk.ndx to generate 
# a new pdb file that contains unique chain identifiers 
u_n20PEG_NR = mda.Universe("NoRes_bsapeg/0.9nm_NoRes/0.9nmNoRes_bpeg.pdb", "NoRes_bsapeg/0.9nm_NoRes/nowat_0.9nmNoRes.xtc")

In [10]:
u_n20PEG_NR

<Universe with 9665 atoms>

Check that we are on the first frame

In [11]:
u_n20PEG_NR.trajectory.frame

0

In [12]:
pn20_lenNR = len(u_n20PEG_NR.trajectory)
pn20_lenNR

10001

In [15]:
# Select one polymer chain, heavy atoms only 
#all_pn20 = u_pn20.select_atoms("(resname sPLG PLG tPLG and segid B) and not type H")

#Select all the PLGA residues, heavy atoms only
all_n20PEG_NR = u_n20PEG_NR.select_atoms("resname sPEG PEG tPEG and not type H")
#list(all_n20PEG_NR)

In [16]:
# Select BSA residues, heavy atoms only 
prot_09nmNR = u_n20PEG_NR.select_atoms("protein and not type H")
prot_09nmNR

<AtomGroup with 4653 atoms>

Calculate AA frame counts for PLGA residues, 1.2 nm RG restraint, 100ns trajectory 

In [17]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20_lenNR - 1
s_time = timeit.default_timer()
h2di_09nmNR = aa_frmcount(prot_09nmNR, all_n20PEG_NR, dmax, u_n20PEG_NR, start, end)
timeit.default_timer() - s_time

278.14253970514983

In [18]:
len(h2di_09nmNR.keys())

207

In [21]:
pr_res_PEG = list(prot_09nmNR.residues)
ss_res_PEG = [str(row) for row in pr_res_PEG]
rkg_NR = {key:h2di_09nmNR[key][1] for key, value in h2di_09nmNR.items()}
plg_09nmNR_occ = pd.DataFrame(data=ss_res_PEG, columns=["BSA_des_res"])
plg_09nmNR_occ['mda_occ_0.9nmNR'] = plg_09nmNR_occ['BSA_des_res'].map(rkg_NR)
plg_09nmNR_occ['mda_occ_0.9nmNR'] = plg_09nmNR_occ['mda_occ_0.9nmNR'].replace('nan', np.nan).fillna(0)
plg_09nmNR_occ['mda_occ_0.9nmNR'] = plg_09nmNR_occ['mda_occ_0.9nmNR'].round(2)
plg_09nmNR_occ

,BSA_des_res,mda_occ_0.9nmNR
0,"<Residue ASP, 1>",0.01
1,"<Residue THR, 2>",0.02
2,"<Residue HIS, 3>",0.00
3,"<Residue LYS, 4>",0.00
4,"<Residue SER, 5>",0.00
...,...,...
578,"<Residue GLN, 579>",0.02
579,"<Residue THR, 580>",0.12
580,"<Residue ALA, 581>",0.18
581,"<Residue LEU, 582>",0.07


In [22]:
plg_09nmNR_occ['mda_occ_0.9nmNR'][plg_09nmNR_occ['mda_occ_0.9nmNR'] != 0]

0      0.01
1      0.02
11     0.01
12     0.01
31     0.04
       ... 
578    0.02
579    0.12
580    0.18
581    0.07
582    0.15
Name: mda_occ_0.9nmNR, Length: 142, dtype: float64

In [23]:
bsa_NR = np.array(list(prot_09nmNR.resids)) # shape is 4652
m_occ_09NR = np.array(list(plg_09nmNR_occ['mda_occ_0.9nmNR'])) # shape is 583
m_occ = np.zeros(shape=(4653))
at_ind = np.where(bsa_NR[:-1] != bsa_NR[1:])[0]
at_in_nw = np.sort(np.append([0,4653],at_ind))
nw_v = 0
for i in range(583):
    b = at_in_nw[i+1] +1
    m_occ[nw_v:b] = m_occ_09NR[i]
    nw_v = at_in_nw[i+1] + 1 

In [25]:
np.nonzero(m_occ)

(array([   0,    1,    2, ..., 4650, 4651, 4652]),)

### Visualize Occupanct on protein 

In [26]:
prot_09nmNR.occupancies = m_occ
prot_09nmNR.occupancies

array([0.01, 0.01, 0.01, ..., 0.15, 0.15, 0.15])

In [27]:
with mda.Writer("prot09nm_NoRes.pdb") as pdb:
    pdb.write(prot_09nmNR)

In [28]:
# Frame count and occupancy for each residue
#h2di

### Residue Importance: 0.9 nm unrestrained 

In [29]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trjmap_09nmPEGNR = prot_poly_cntmovie(prot_09nmNR, all_n20PEG_NR, dmax, u_n20PEG_NR, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [30]:
np.save('0.9nmPEG_NoRes.npy', trjmap_09nmPEGNR)    # .npy extension is added if not given

In [ ]:
trjmap_09nmPEG = np.load("1.2nm_res.npy", allow_pickle=True)

In [31]:
trjmap_09nmPEGNR[0].shape

(583, 60)

In [32]:
np.sum(trjmap_09nmPEGNR[1000][0])

0.0

In [33]:
kj = np.zeros(shape=(10000, 583))
kj[:,582].shape

(10000,)

In [34]:
pp_09nmNR_ct = np.zeros(shape=(10000, 583))
for i in range(10000):
    for j in range(583):
        pp_09nmNR_ct[i][j] = np.sum(trjmap_09nmPEGNR[i][j])

In [35]:
pp_09nmtotNR = np.zeros(shape=(583))
for i in range(583):
    pp_09nmtotNR[i] = np.sum(pp_09nmNR_ct[:,i])
#pp_12nmtot

In [36]:
np.sum(pp_09nmtotNR)

400424.0

In [41]:
y_pos = np.arange(583) + 1
wid = np.zeros(shape=583)
wid += 2
#wid
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(y_pos, pp_09nmtotNR, align='center',width=wid, alpha=1, label='0.9 nm PEG')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,600])
plt.ylim([0,80000])
plt.legend(fontsize=14)
plt.ylabel(r'Total No. of PEG contacts', fontsize=15)
plt.xlabel(r'BSA Residue ID', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'BSA Residue ID')

### Total number of residues that are within 4 angstroms of a PEG oligomer residue within a 100 ns trajectory block

In [38]:
a_a = ["GLY","ALA","VAL","LEU","ILE","MET","PHE","TRP","PRO","SER","THR","CYS","TYR","ASN","GLN","ASP"
               ,"GLU","LYS","ARG","HIS"]

In [42]:
# This code chunk gets the BSA residues and their corresponding number in a pandas dataframe 
red_bsa = []
bh = np.arange(0,584)
for i in range(583):
    b_str = str(list(prot_09nmNR.residues[i:i+1]))
    if str(bh[i+1]) in b_str: 
        red_bsa.append(str(b_str[10:13])+" "+str(bh[i+1]))

In [43]:
pr_res = list(prot_09nmNR.residues)
ss_res = [str(row) for row in pr_res]
rkg = {key:h2di_09nmNR[key][0] for key, value in h2di_09nmNR.items()}
plg_09nmaaNR = pd.DataFrame(data=ss_res, columns=["BSA_des_res"])
plg_09nmaaNR['mda_plga_frm_0.9nmNR'] = plg_09nmaaNR['BSA_des_res'].map(rkg)
plg_09nmaaNR['BSA_des_res'] = red_bsa
plg_09nmaaNR['mda_plga_frm_0.9nmNR'] = plg_09nmaaNR['mda_plga_frm_0.9nmNR'].replace('nan', np.nan).fillna(0)
plg_09nmaaNR.tail()

,BSA_des_res,mda_plga_frm_0.9nmNR
578,GLN 579,220.0
579,THR 580,1215.0
580,ALA 581,1836.0
581,LEU 582,684.0
582,ALA 583,1522.0


In [44]:
# Read in data from the oputput of wrapper.sh, where the frame count is given for each BSA residue that was within 
# 4 angstroms of PLGA trimer 
wat_data = pd.read_csv('occ_BSA1ns.txt', sep=" ", header=None, usecols=None ,index_col=None)
wat_data.columns = ["BSA_res_no","No. of frames (VMD)"]
wat_data = wat_data.drop("BSA_res_no", axis=1)

#pr_res = list(prot_09nm.residues)
#ss_res = [str(row) for row in pr_res]

wat_data['BSA_des_res'] = ss_res
wat_data = wat_data[['BSA_des_res',"No. of frames (VMD)"]]
#wat_data.head()

# load MDAnalysis values from MDA_BSA1ns.txt file(129003 atoms SOL group was used to calc. frame counts for txt.
# file)
h2ob_dict = json.load(open("MDA_BSA1ns.txt"))
wat_data['Mda_frames'] = wat_data['BSA_des_res'].map(h2ob_dict)


# From MD Analysis
#Get the count of bsa residues that have 1001 or 1002 frames ( I ran a 1 ns NPT simulation of 1 BSA in water )

#aa_count = pd.DataFrame(data=a_a)
c_list = []

for i in range(len(a_a)):
    count = 0
    for index, row in wat_data.iterrows():
        if a_a[i] in row["BSA_des_res"]:
            if row['Mda_frames'] == 1001: 
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames']))) 
            elif row['Mda_frames'] == 1000:
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames'])))
    c_list.append(str(str(a_a[i])+"  "+str(count)))
    
#c_list

# From VMD
#Get the count of bsa residues that have 1001 or 1002 frames ( I ran a 1 ns NPT simulation of 1 BSA in water )

#aa_count = pd.DataFrame(data=a_a)
vmd_list = []

for i in range(len(a_a)):
    count = 0
    for index, row in wat_data.iterrows():
        if a_a[i] in row["BSA_des_res"]:
            if row["No. of frames (VMD)"] == 1001: 
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames']))) 
            elif row["No. of frames (VMD)"] == 1002:
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames'])))
    vmd_list.append(str(str(a_a[i])+"  "+str(count)))

# Main difference is that Alanine 583 is counted for all 1001 frames. It seems VMD is unable to calc dist for that res
#vmd_list

#hydrophobic_res = ['ALA', 'ILE', 'LEU', 'VAL', 'GLY', 'PRO','PHE', 'TRP','MET']
#polar_res = ['ASN', 'CYS', 'GLN', 'SER', 'THR','TYR']
#neg_res = ['ASP', 'GLU']
#pos_res = ['ARG', 'HIS', 'LYS']
# aromatic_res = ['PHE', 'TRP', 'TYR', 'HIS']
#all_res = [pos_res, neg_res, polar_res, hydrophobic_res]

# Put the AA count in a pandas dataframe 
dg , ji = AA_list_org(c_list)
aa_count = pd.DataFrame(data=dg, index=None, columns=['Amino_acids'])
new_lf = pd.Series(data=ji, index=None)
vmg, vmdj = AA_list_org(vmd_list)
n2lf = pd.Series(data=vmdj, index=None)
aa_count['No_of_surf_res (MDAnalysis)'] = new_lf
aa_count['No_of_surf_res (VMD)'] = n2lf

In [45]:
apl_list = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plg_09nmaaNR.iterrows():
    if r_pl['mda_plga_frm_0.9nmNR'] != 0:
        apl_list.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_l = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_list)):
        if r_a['Amino_acids'] in apl_list[i]:
            count += 1
    cpl_l.append(count)      
        
aa_count['peg_0.9nmNR_100ns'] = cpl_l
aa_count

,Amino_acids,No_of_surf_res (MDAnalysis),No_of_surf_res (VMD),peg_0.9nmNR_100ns
0,LYS,59,59,34
1,ARG,23,23,7
2,HIS,16,16,8
3,ASP,40,40,22
4,GLU,59,59,29
5,SER,23,23,9
6,THR,29,29,16
7,CYS,24,24,11
8,TYR,19,19,5
9,ASN,14,14,5


In [46]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['peg_0.9nmNR_100ns'].sum()

207

In [47]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

487

In [48]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['peg_0.9nmNR_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

0.42505133470225875

Calculate mean occupancy and the standard deviation for 1.2 nm trajectory

Numpy mean and std function was used to calculate mean occupancy and std dev using occ values from aa_frmcount output

In [49]:
# Mean occupancy and std deviation 
ll_mo = [value[1] for key, value in h2di_09nmNR.items()]
print("Mean Occpancy (1.2 nm Rg): "+str(np.mean(ll_mo)), "Occ. std. dev.: "+str(np.std(ll_mo)))

Mean Occpancy (1.2 nm Rg): 0.08166376811594205 Occ. std. dev.: 0.154123690809989


### Calc. fractional contacts for each AA group type 

In [50]:
cd_12nm = frac_cont(h2di_09nmNR)
cd_12nm

{'Negative': [42994.0, 51, 174505.0588235294, 0.23026960477380862],
 'Positive': [42291.0, 49, 178657.89795918367, 0.2357495182668802],
 'Polar': [42006.0, 55, 158095.30909090908, 0.20861598274793475],
 'Hydrophobic': [41753.0, 52, 166209.0576923077, 0.21932254733852058],
 'Aromatic': [6988.0, 18, 80362.0, 0.10604234687285578]}

In [51]:
cd = frac_cont(h2di_09nmNR)
kklh = []
for key, value in cd.items():
    kklh.append(value[1])
# Must substract aromatic residues, since they are already counted
sum(kklh) - cd['Aromatic'][1]

207

In [ ]:
start = 0
end = pn20_len - 1
aa_12nm, l_f12nm = gtwo_trjcnt(prot, all_pn20, dmax, u_pn20, start, end)

In [ ]:
aa_12nm

In [ ]:
l_f12nm.shape

In [52]:
no_surf = aa_count['No_of_surf_res (MDAnalysis)'].sum()
no_surf

487

In [53]:
fcnt_rg09nmNR, prgrp_09nmNR, aa_matx09nmNR = bavg_frac_cnt(5, prot_09nmNR, all_n20PEG_NR, dmax, u_n20PEG_NR, no_surf, 0, 10000)

0
2000
2000
4000
4000
6000
6000
8000
8000
10000


In [54]:
fcnt_rg09nmNR

{'Negative': array([0.20664114, 0.31724495, 0.20979678, 0.227986  , 0.21781872]),
 'Positive': array([0.25340817, 0.24748282, 0.25531048, 0.2151241 , 0.25491458]),
 'Polar': array([0.22462771, 0.24642146, 0.15208788, 0.18901906, 0.2444011 ]),
 'Hydrophobic': array([0.2633488 , 0.12071504, 0.20178445, 0.20977559, 0.20805935]),
 'Aromatic': array([0.05197418, 0.06813573, 0.18102042, 0.15809525, 0.07480625]),
 'total_frac': array([0.19507187, 0.16837782, 0.2073922 , 0.19507187, 0.16837782])}

In [56]:
fc09nm_meanNR = np.array([np.mean(fcnt_rg09nmNR['Negative']), np.mean(fcnt_rg09nmNR['Positive'])
                        ,np.mean(fcnt_rg09nmNR['Polar']),np.mean(fcnt_rg09nmNR['Hydrophobic'])
                        , np.mean(fcnt_rg09nmNR['Aromatic'])])
fc09nm_meanNR

array([0.23589752, 0.24524803, 0.21131144, 0.20073665, 0.10680636])

In [57]:
fc09nm_stdNR = np.array([np.std(fcnt_rg09nmNR['Negative']), np.std(fcnt_rg09nmNR['Positive'])
                       ,np.std(fcnt_rg09nmNR['Polar']),np.std(fcnt_rg09nmNR['Hydrophobic'])
                       , np.std(fcnt_rg09nmNR['Aromatic'])])
fc09nm_stdNR

array([0.04133769, 0.01532141, 0.03608142, 0.04573874, 0.05227671])

In [59]:
x_pos = np.arange(5)
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '500px'
fig.canvas.layout.height = '400px'
plt.bar(x_pos, fc09nm_meanNR, yerr=fc09nm_stdNR, ecolor='black',capsize=5, color='c')
plt.title(r'Fractional Contacts 0.9 nm Rg restrained', fontsize=15)
plt.xticks(x_pos, labels=aa_types, fontsize=12)
plt.ylabel(r'Fractional Contacts', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Fractional Contacts')

### Total fraction of contacts: averages and std dev calc from 5 20ns blocks

In [60]:
# Average of total fraction of contacts
np.mean(fcnt_rg09nmNR['total_frac'])

0.1868583162217659

In [61]:
# Std Deviation of total fraction of contacts
np.std(fcnt_rg09nmNR['total_frac'])

0.01574561766955303

### Avg no. PLGA residues per BSA AA residue group 

In [ ]:
prgrp_1_2nm

In [ ]:
mean_12nm = np.zeros(shape=5)
std_12nm = np.zeros(shape=5)
count = 0
for key, value in prgrp_1_2nm.items():
    mpl_12nm = []
    var_12nm = []
    for i in prgrp_1_2nm[str(key)].flat:
        mpl_12nm.append(i[0])
        var_12nm.append((i[1])**2)
    
    # calc frac cont averages
    mean_12nm[count] = np.mean(mpl_12nm)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_12nm[count] = np.std(mpl_12nm)
    # std_12nm[count] = np.sqrt(np.sum(var_12nm)/5)
    
    count += 1


In [ ]:
mean_12nm

In [ ]:
std_12nm 

In [ ]:
x_pos = np.arange(5)
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, mean_12nm, yerr=std_12nm, ecolor='black',capsize=5)
plt.title(r'No. of PLGA residues 1.2 nm Rg restrained', fontsize=15)
plt.xticks(x_pos, labels=aa_types, fontsize=12)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

### Protein/polymer contact map movie

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 1.2 nm res", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.93, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_12nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('1.2nm_res.mp4',writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()

# 1.1 nm PEG unrestrained Rg 100 ns trajectory

Load the rg = 1.5 nm (3 PLGA N = 20 oligomer/BSA system) 

In [62]:
# Set up the MD Simulation
u11nmNR_n20PEG = mda.Universe("NoRes_bsapeg/1.1nm_NoRes/1.1nmNoRes_bpeg.pdb", "NoRes_bsapeg/1.1nm_NoRes/nowat_1.1nmNoRes.xtc")

In [63]:
u11nmNR_n20PEG

<Universe with 9665 atoms>

In [64]:
pn20_11nmNR = len(u11nmNR_n20PEG.trajectory)
pn20_11nmNR

10001

In [65]:
#Select all the PLGA residues, heavy atoms only 
n20PEG_11nmNR = u11nmNR_n20PEG.select_atoms("resname sPEG PEG tPEG and not type H")
n20PEG_11nmNR

<AtomGroup with 183 atoms>

In [66]:
# Select BSA residues, heavy atoms only 
protNR_11nmPEG = u11nmNR_n20PEG.select_atoms("protein and not type H")
protNR_11nmPEG

<AtomGroup with 4653 atoms>

### Contact Analysis

In [67]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20_11nmNR - 1
s_time = timeit.default_timer()
h2di_11nmNR = aa_frmcount(protNR_11nmPEG, n20PEG_11nmNR, dmax, u11nmNR_n20PEG, start, end)
timeit.default_timer() - s_time
#h2di_11nm

478.9605230214074

In [68]:
len(h2di_11nmNR.keys())

194

In [69]:
pr_res_PEG = list(protNR_11nmPEG.residues)
ss_res_PEG = [str(row) for row in pr_res_PEG]
rkg_11NR = {key:h2di_11nmNR[key][1] for key, value in h2di_11nmNR.items()}
plg_09nmNR_occ['mda_occ_1.1nmNR'] = plg_09nmNR_occ['BSA_des_res'].map(rkg_11NR)
plg_09nmNR_occ['mda_occ_1.1nmNR'] = plg_09nmNR_occ['mda_occ_1.1nmNR'].replace('nan', np.nan).fillna(0)
plg_09nmNR_occ['mda_occ_1.1nmNR'] = plg_09nmNR_occ['mda_occ_1.1nmNR'].round(2)
plg_09nmNR_occ

,BSA_des_res,mda_occ_0.9nmNR,mda_occ_1.1nmNR
0,"<Residue ASP, 1>",0.01,0.0
1,"<Residue THR, 2>",0.02,0.0
2,"<Residue HIS, 3>",0.00,0.0
3,"<Residue LYS, 4>",0.00,0.0
4,"<Residue SER, 5>",0.00,0.0
...,...,...,...
578,"<Residue GLN, 579>",0.02,0.0
579,"<Residue THR, 580>",0.12,0.0
580,"<Residue ALA, 581>",0.18,0.0
581,"<Residue LEU, 582>",0.07,0.0


In [72]:
pr_res11nm = list(protNR_11nmPEG.residues)
ss_res11nm = [str(row) for row in pr_res11nm]
rkg_11nmNR = {key:h2di_11nmNR[key][0] for key, value in h2di_11nmNR.items()}
plg_1_1nmaaNR = pd.DataFrame(data=ss_res11nm, columns=["BSA_des_res"])
plg_1_1nmaaNR['mda_plga_frm_1.1nmNR'] = plg_1_1nmaaNR['BSA_des_res'].map(rkg_11nmNR)
plg_1_1nmaaNR['BSA_des_res'] = red_bsa
plg_1_1nmaaNR['mda_plga_frm_1.1nmNR'] = plg_1_1nmaaNR['mda_plga_frm_1.1nmNR'].replace('nan', np.nan).fillna(0)
plg_1_1nmaaNR.head()

,BSA_des_res,mda_plga_frm_1.1nmNR
0,ASP 1,0.0
1,THR 2,0.0
2,HIS 3,0.0
3,LYS 4,0.0
4,SER 5,0.0


In [73]:
bsa_NR = np.array(list(protNR_11nmPEG.resids)) # shape is 4652
m_occ_11nmNR = np.array(list(plg_09nmNR_occ['mda_occ_1.1nmNR'])) # shape is 583
m_occ_11New = np.zeros(shape=(4653))
at_ind = np.where(bsa_NR[:-1] != bsa_NR[1:])[0]
at_in_nw = np.sort(np.append([0,4653],at_ind))
nw_v = 0
for i in range(583):
    b = at_in_nw[i+1] +1
    m_occ_11New[nw_v:b] = m_occ_11nmNR[i]
    nw_v = at_in_nw[i+1] + 1 

In [74]:
m_occ_11New[0:33]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
protNR_11nmPEG.occupancies[3089:3099]

In [75]:
np.nonzero(m_occ_11New)

(array([ 257,  258,  259, ..., 4478, 4479, 4480]),)

### Visualize Occupanct on protein 

In [76]:
protNR_11nmPEG.occupancies = m_occ_11New
protNR_11nmPEG.occupancies[0:33]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [77]:
with mda.Writer("prot11nm_NoRes.pdb") as pdb:
    pdb.write(protNR_11nmPEG)

### Residue Importance: 1.1 nm unrestrained 

In [78]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trjmap_11nmPEGNR = prot_poly_cntmovie(protNR_11nmPEG, n20PEG_11nmNR, dmax, u11nmNR_n20PEG, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [ ]:
trj_ppmap_15nm = np.load("1.5nm_res.npy", allow_pickle=True)

In [79]:
np.save('1.1nmNR_PEGres.npy', trjmap_11nmPEGNR)    # .npy extension is added if not given

In [80]:
trjmap_11nmPEGNR[0].shape

(583, 60)

In [81]:
np.sum(trjmap_11nmPEGNR[1000][0])

0.0

In [82]:
kj = np.zeros(shape=(10000, 583))
kj[:,582].shape

(10000,)

In [83]:
pp_11nmNR_ct = np.zeros(shape=(10000, 583))
for i in range(10000):
    for j in range(583):
        pp_11nmNR_ct[i][j] = np.sum(trjmap_11nmPEGNR[i][j])

In [84]:
pp_11nmtotNR = np.zeros(shape=(583))
for i in range(583):
    pp_11nmtotNR[i] = np.sum(pp_11nmNR_ct[:,i])
#pp_12nmtot

In [85]:
np.sum(pp_11nmtotNR)

813539.0

In [87]:
y_pos = np.arange(583) + 1
wid = np.zeros(shape=583)
wid += 1.5
#wid
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(y_pos+0.25, pp_11nmtotNR, align='center',width=wid, color='#562A8B', alpha=0.3, label='1.1 nm PEG')
plt.bar(y_pos, pp_09nmtotNR, align='center',width=wid, alpha=0.5, color='#1D77CF',label='0.9 nm PEG')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,600])
plt.ylim([0,80000])
plt.legend(fontsize=14)
plt.ylabel(r'Total No. of PEG contacts', fontsize=15)
plt.xlabel(r'BSA Residue ID', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'BSA Residue ID')

### Total number of residues that are within 4 angstroms of a PEG oligomer residue within a 100 ns trajectory block

In [88]:
apl_15nmNR = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plg_1_1nmaaNR.iterrows():
    if r_pl['mda_plga_frm_1.1nmNR'] != 0:
        apl_15nmNR.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_15nmNR = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_15nmNR)):
        if r_a['Amino_acids'] in apl_15nmNR[i]:
            count += 1
    cpl_15nmNR.append(count)      
        
aa_count['peg_1.1nmNR_100ns'] = cpl_15nmNR
#aa_count.drop('No_of_surf_res (VMD)', axis=1, inplace=True)
aa_count

,Amino_acids,No_of_surf_res (MDAnalysis),No_of_surf_res (VMD),peg_0.9nmNR_100ns,peg_1.1nmNR_100ns
0,LYS,59,59,34,26
1,ARG,23,23,7,10
2,HIS,16,16,8,3
3,ASP,40,40,22,16
4,GLU,59,59,29,23
5,SER,23,23,9,6
6,THR,29,29,16,16
7,CYS,24,24,11,8
8,TYR,19,19,5,6
9,ASN,14,14,5,4


In [89]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['peg_1.1nmNR_100ns'].sum()

194

In [90]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

487

In [91]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['peg_1.1nmNR_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

0.39835728952772076

In [92]:
# Mean occupancy and std deviation 
ll_mo15 = [value[1] for key, value in h2di_11nmNR.items()]
print("Mean Occpancy (1.1 nm Rg): "+str(np.mean(ll_mo15)), "Occ. std. dev.: "+str(np.std(ll_mo15)))

Mean Occpancy (1.1 nm Rg): 0.2016030927835052 Occ. std. dev.: 0.2688160615344409


In [93]:
cd_11nmNR = frac_cont(h2di_11nmNR)
cd_11nmNR

{'Negative': [82535.0, 39, 410558.71794871794, 0.20327846612230716],
 'Positive': [83988.0, 39, 417786.4615384615, 0.2068571128936855],
 'Polar': [85857.0, 50, 333125.16000000003, 0.1649390661825054],
 'Hydrophobic': [138730.0, 66, 407782.1212121212, 0.20190369973450367],
 'Aromatic': [39471.0, 17, 450433.7647058824, 0.22302165506699823]}

### Calc. fractional contacts for each AA group type 

In [94]:
fcnt_rg11nmNR, prgrp_11nmNR, aamatx_11nmNR = bavg_frac_cnt(5, protNR_11nmPEG, n20PEG_11nmNR, dmax ,u11nmNR_n20PEG, no_surf, 0, 10000)

0
2000
2000
4000
4000
6000
6000
8000
8000
10000


In [95]:
fcnt_rg11nmNR

{'Negative': array([0.20300842, 0.23379024, 0.18786607, 0.19865067, 0.19501947]),
 'Positive': array([0.20365344, 0.17572495, 0.20500498, 0.23368864, 0.25536564]),
 'Polar': array([0.20050905, 0.14371756, 0.15930811, 0.16313209, 0.18437636]),
 'Hydrophobic': array([0.19747625, 0.23069797, 0.21778499, 0.20142137, 0.17663019]),
 'Aromatic': array([0.19535285, 0.21606927, 0.23003586, 0.20310723, 0.18860834]),
 'total_frac': array([0.26488706, 0.26283368, 0.24229979, 0.26078029, 0.2238193 ])}

In [96]:
fc11nm_meanNR = np.array([np.mean(fcnt_rg11nmNR['Negative']), np.mean(fcnt_rg11nmNR['Positive'])
                        ,np.mean(fcnt_rg11nmNR['Polar']),np.mean(fcnt_rg11nmNR['Hydrophobic'])
                        , np.mean(fcnt_rg11nmNR['Aromatic'])])
fc11nm_meanNR

array([0.20366697, 0.21468753, 0.17020863, 0.20480215, 0.20663471])

In [97]:
fc11nm_stdNR = np.array([np.std(fcnt_rg11nmNR['Negative']), np.std(fcnt_rg11nmNR['Positive'])
                       ,np.std(fcnt_rg11nmNR['Polar']),np.std(fcnt_rg11nmNR['Hydrophobic'])
                       , np.std(fcnt_rg11nmNR['Aromatic'])])
fc11nm_stdNR

array([0.01585848, 0.0273837 , 0.01994751, 0.01842773, 0.01484115])

In [99]:
x_pos = np.arange(5)
width = 0.35
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(x_pos, fc09nm_meanNR, width, yerr=fc09nm_stdNR, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, fc11nm_meanNR, width, yerr=fc11nm_stdNR, ecolor='black',capsize=5, color='c')
plt.title(r'Fractional Contacts Rg restrained', fontsize=15)
plt.xticks(x_pos+width/2, labels=aa_types, fontsize=12)
plt.legend(['Rg = 0.9 nm', 'Rg = 1.1 nm'], frameon=False)
plt.ylabel(r'Fractional Contacts', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Fractional Contacts')

### Total fraction of contacts: averages and std dev calc from 5 20 ns blocks

In [100]:
np.mean(fcnt_rg11nmNR['total_frac'])

0.2509240246406571

In [101]:
np.std(fcnt_rg11nmNR['total_frac'])

0.015767025725217633

In [ ]:
prgrp_11nmNR

In [ ]:
# matrix containing the avg number of PLGA residues for each block for each amino acid 
np.where(aamatx_15nmNR[0, 0] != 0)

### Avg no. PLGA residues per BSA AA residue group 

In [ ]:
prgrp_1_5nm

In [ ]:
mean_15nm = np.zeros(shape=5)
std_15nm = np.zeros(shape=5)
count = 0
for key, value in prgrp_1_5nm.items():
    mpl_15nm = []
    var_15nm = []
    for i in prgrp_1_5nm[str(key)].flat:
        mpl_15nm.append(i[0])
        var_15nm.append((i[1])**2)
    
    # calc frac cont averages
    mean_15nm[count] = np.mean(mpl_15nm)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_15nm[count] = np.std(mpl_15nm)
    #std_15nm[count] = np.sqrt(np.sum(var_15nm)/5)
    
    count += 1


In [ ]:
mean_15nm

In [ ]:
std_15nm

In [ ]:
x_pos = np.arange(5)
width = 0.35
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, mean_12nm, width, yerr=std_12nm, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, mean_15nm, width, yerr=std_15nm, ecolor='black',capsize=5, color='c')
plt.title(r'No. of PLGA residues Rg restrained', fontsize=15)
plt.xticks(x_pos+width/2, labels=aa_types, fontsize=12)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm'], frameon=False)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

### Protein/polymer contact map movie

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 1.5 nm res.", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.94, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_15nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1.03, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('1.5nm_res.mp4', writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()

# 1.4 nm unrestrained Rg PEG 100 ns trajectory

### Contact Analyis

In [102]:
# Set up the MD Simulation
u14nmNR_n20PEG = mda.Universe("NoRes_bsapeg/1.4nm_NoRes/1.4nmNoRes_bpeg.pdb", "NoRes_bsapeg/1.4nm_NoRes/nowat_1.4nmNoRes.xtc")

In [103]:
u14nmNR_n20PEG

<Universe with 9665 atoms>

In [104]:
pn20_len14nmNR = len(u14nmNR_n20PEG.trajectory)
pn20_len14nmNR

10001

In [105]:
#Select all the PLGA residues, heavy atoms only 
pn20_14nmNR = u14nmNR_n20PEG.select_atoms("resname sPEG PEG tPEG and not type H")
pn20_14nmNR

<AtomGroup with 183 atoms>

In [106]:
# Select BSA residues, heavy atoms only 
prot_14nmNR = u14nmNR_n20PEG.select_atoms("protein and not type H")
prot_14nmNR

<AtomGroup with 4653 atoms>

In [107]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20_len14nmNR - 1
s_time = timeit.default_timer()
h2di_14nmNR = aa_frmcount(prot_14nmNR, pn20_14nmNR, dmax, u14nmNR_n20PEG, start, end)
timeit.default_timer() - s_time
#h2di_14nm

383.2374846050516

In [109]:
len(h2di_14nmNR.keys())

150

In [111]:
pr_res14nmNR = list(prot_14nmNR.residues)
ss_res14nmNR = [str(row) for row in pr_res14nmNR]
rkg_14nmNR = {key:h2di_14nmNR[key][0] for key, value in h2di_14nmNR.items()}
plg_14nmaaNR = pd.DataFrame(data=ss_res14nmNR, columns=["BSA_des_res"])
plg_14nmaaNR['mda_plga_frm_1.4nmNR'] = plg_14nmaaNR['BSA_des_res'].map(rkg_14nmNR)
plg_14nmaaNR['BSA_des_res'] = red_bsa
plg_14nmaaNR['mda_plga_frm_1.4nmNR'] = plg_14nmaaNR['mda_plga_frm_1.4nmNR'].replace('nan', np.nan).fillna(0)
plg_14nmaaNR.head()

,BSA_des_res,mda_plga_frm_1.4nmNR
0,ASP 1,62.0
1,THR 2,1857.0
2,HIS 3,6243.0
3,LYS 4,370.0
4,SER 5,1421.0


In [113]:
# Extract mean occupancy values
pr_res_14ur = list(prot_14nmNR.residues)
ss_res_14ur = [str(row) for row in pr_res_14ur]
rkg_14NR = {key:h2di_14nmNR[key][1] for key, value in h2di_14nmNR.items()}
plg_09nmNR_occ['mda_occ_1.4nmNR'] = plg_09nmNR_occ['BSA_des_res'].map(rkg_14NR)
plg_09nmNR_occ['mda_occ_1.4nmNR'] = plg_09nmNR_occ['mda_occ_1.4nmNR'].replace('nan', np.nan).fillna(0)
plg_09nmNR_occ['mda_occ_1.4nmNR'] = plg_09nmNR_occ['mda_occ_1.4nmNR'].round(2)
plg_09nmNR_occ

,BSA_des_res,mda_occ_0.9nmNR,mda_occ_1.1nmNR,mda_occ_1.4nmNR
0,"<Residue ASP, 1>",0.01,0.0,0.01
1,"<Residue THR, 2>",0.02,0.0,0.19
2,"<Residue HIS, 3>",0.00,0.0,0.62
3,"<Residue LYS, 4>",0.00,0.0,0.04
4,"<Residue SER, 5>",0.00,0.0,0.14
...,...,...,...,...
578,"<Residue GLN, 579>",0.02,0.0,0.00
579,"<Residue THR, 580>",0.12,0.0,0.03
580,"<Residue ALA, 581>",0.18,0.0,0.00
581,"<Residue LEU, 582>",0.07,0.0,0.00


In [115]:
bsa_NR = np.array(list(prot_14nmNR.resids)) # shape is 4652
m_occ_14nmNR = np.array(list(plg_09nmNR_occ['mda_occ_1.4nmNR'])) # shape is 583
m_occ_14New = np.zeros(shape=(4653))
at_ind = np.where(bsa_NR[:-1] != bsa_NR[1:])[0]
at_in_nw = np.sort(np.append([0,4653],at_ind))
nw_v = 0
for i in range(583):
    b = at_in_nw[i+1] +1
    m_occ_14New[nw_v:b] = m_occ_14nmNR[i]
    nw_v = at_in_nw[i+1] + 1 

In [116]:
np.nonzero(m_occ_14New)

(array([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
          11,   12,   13,   14,   15,   16,   17,   18,   19,   20,   21,
          22,   23,   24,   25,   26,   27,   28,   29,   30,   31,   32,
          33,   34,   35,   36,   37,   38,   39,   57,   58,   59,   60,
          61,   62,   63,   64,   65,   66,   67,   68,   69,   70,   71,
          83,   84,   85,   86,   87,   88,   89,   90,   91,   92,   93,
          94,   95,   96,   97,   98,   99,  100,  101,  102,  103,  104,
         105,  106,  107,  108,  109,  110,  111,  112,  113,  114,  115,
         116,  117,  118,  119,  120,  121,  122,  123,  124,  125,  126,
         127,  128,  129,  130,  131,  132,  133,  134,  135,  136,  137,
         138,  139,  140,  141,  142,  143,  144,  145,  146,  147,  148,
         149,  150,  266,  267,  268,  269,  270,  271,  272,  273,  274,
         275,  276,  277,  278,  279,  280,  281,  282,  283,  284,  285,
         286,  287,  414,  415,  416, 

### Visualize Occupanct on protein 

In [117]:
prot_14nmNR.occupancies = m_occ_14New
prot_14nmNR.occupancies

array([0.01, 0.01, 0.01, ..., 0.  , 0.  , 0.  ])

In [118]:
with mda.Writer("prot14nm_NoRes.pdb") as pdb:
    pdb.write(prot_14nmNR)

### Residue Importance: 1.4 nm unrestrained 

In [119]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trjmap_14nmPEGNR = prot_poly_cntmovie(prot_14nmNR, pn20_14nmNR, dmax, u14nmNR_n20PEG, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [120]:
np.save('1.4nm_NoRes.npy', trjmap_14nmPEGNR)    # .npy extension is added if not given

In [ ]:
trj_ppmap_2nm = np.load('2nm_res.npy', allow_pickle=True)
trj_ppmap_2nm

In [121]:
trjmap_14nmPEGNR[0].shape

(583, 60)

In [122]:
np.sum(trjmap_14nmPEGNR[1000][0])

0.0

In [123]:
kj = np.zeros(shape=(10000, 583))
kj[:,582].shape

(10000,)

In [124]:
pp_14nmNR_ct = np.zeros(shape=(10000, 583))
for i in range(10000):
    for j in range(583):
        pp_14nmNR_ct[i][j] = np.sum(trjmap_14nmPEGNR[i][j])

In [125]:
pp_14nmtotNR = np.zeros(shape=(583))
for i in range(583):
    pp_14nmtotNR[i] = np.sum(pp_14nmNR_ct[:,i])
#pp_12nmtot

In [127]:
np.nonzero(pp_14nmtotNR)

(array([  0,   1,   2,   3,   4,   7,   8,  10,  11,  12,  13,  14,  15,
         16,  17,  19,  32,  33,  34,  35,  36,  37,  47,  50,  51,  52,
         53,  54,  55,  56,  77,  78,  81,  82, 109, 110, 111, 112, 113,
        114, 115, 116, 125, 128, 129, 130, 133, 135, 138, 139, 142, 143,
        156, 157, 158, 160, 161, 164, 257, 260, 277, 278, 279, 280, 281,
        282, 283, 284, 318, 346, 349, 350, 353, 354, 357, 358, 359, 360,
        361, 362, 363, 364, 366, 367, 368, 369, 370, 371, 373, 374, 375,
        376, 377, 378, 380, 381, 387, 417, 418, 419, 440, 466, 477, 478,
        481, 483, 484, 487, 488, 496, 497, 498, 499, 500, 501, 502, 503,
        504, 506, 507, 508, 509, 510, 511, 513, 522, 525, 529, 532, 548,
        555, 558, 559, 560, 561, 562, 563, 564, 566, 567, 568, 569, 571,
        572, 573, 575, 576, 578, 579, 580]),)

In [128]:
a_peg = np.sum(pp_14nmtotNR)

In [129]:
b_peg = np.sum(pp_11nmtotNR)

In [130]:
c_peg = np.sum(pp_09nmtotNR)

In [ ]:
plt.close('all')

In [131]:
y_pos = np.arange(583) + 1
wid = np.zeros(shape=583)
wid += 1.5
#wid
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(y_pos, pp_09nmtotNR/c_peg, align='center',width=wid, alpha=0.5, color='#1D77CF',label='0.9 nm PEG')
plt.bar(y_pos+0.25, pp_11nmtotNR/b_peg, align='center',width=wid, color='#562A8B', alpha=0.3, label='1.1 nm PEG')
plt.bar(y_pos+0.3, pp_14nmtotNR/a_peg, align='center',width=wid, color='#4E4C4D', alpha=0.3, label='1.4 nm PEG')
#plt.bar(y_pos+0.25, pp_11nmtot/b_peg, align='center',width=wid, color='#562A8B', alpha=0.3, label='1.1 nm PEG')
#plt.bar(y_pos, pp_09nmtot/c_peg, align='center',width=wid, alpha=0.5, color='#1D77CF',label='0.9 nm PEG')
plt.title("BSA in water with PEG Unrestrained, 100 ns", fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,600])
plt.ylim([0,0.2])
plt.legend(fontsize=14)
plt.ylabel(r'Normalized Total No. of PEG contacts', fontsize=15)
plt.xlabel(r'BSA Residue ID', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'BSA Residue ID')

### Total number of residues that are within 4 angstroms of a PEG oligomer residue within a 100 ns trajectory block

In [135]:
apl_14nmNR = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plg_14nmaaNR.iterrows():
    if r_pl['mda_plga_frm_1.4nmNR'] != 0:
        apl_14nmNR.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_14nmNR = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_14nmNR)):
        if r_a['Amino_acids'] in apl_14nmNR[i]:
            count += 1
    cpl_14nmNR.append(count)      
        
aa_count['peg_1.4nmNR_100ns'] = cpl_14nmNR
#aa_count.drop('No_of_surf_res (VMD)', axis=1, inplace=True)
aa_count

,Amino_acids,No_of_surf_res (MDAnalysis),No_of_surf_res (VMD),peg_0.9nmNR_100ns,peg_1.1nmNR_100ns,peg_1.4nmNR_100ns
0,LYS,59,59,34,26,24
1,ARG,23,23,7,10,4
2,HIS,16,16,8,3,5
3,ASP,40,40,22,16,16
4,GLU,59,59,29,23,18
5,SER,23,23,9,6,7
6,THR,29,29,16,16,10
7,CYS,24,24,11,8,6
8,TYR,19,19,5,6,2
9,ASN,14,14,5,4,4


In [136]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['peg_1.4nmNR_100ns'].sum()

150

In [137]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

487

In [138]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['peg_1.4nmNR_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

0.3080082135523614

In [139]:
# Mean occupancy and std deviation 
ll_mo2nm = [value[1] for key, value in h2di_14nmNR.items()]
print("Mean Occpancy (1.4 nm Rg): "+str(np.mean(ll_mo2nm)), "Occ. std. dev.: "+str(np.std(ll_mo2nm)))

Mean Occpancy (1.4 nm Rg): 0.18995399999999998 Occ. std. dev.: 0.29892610026114036


In [140]:
cd_14NRnm = frac_cont(h2di_14nmNR)
cd_14NRnm

{'Negative': [69719.0, 34, 307583.8235294118, 0.17914206531715288],
 'Positive': [74080.0, 33, 336727.2727272727, 0.19611570723324373],
 'Polar': [54180.0, 33, 246272.72727272726, 0.14343343706664613],
 'Hydrophobic': [86952.0, 50, 260856.0, 0.1519269838516159],
 'Aromatic': [52784.0, 14, 565542.8571428572, 0.32938180653134136]}

### Calc. fractional contacts for each AA group type 

In [141]:
fcnt_rg14nmNR, prgrp_14nmNR, aa_matx_14nmNR = bavg_frac_cnt(5, prot_14nmNR, pn20_14nmNR, dmax, u14nmNR_n20PEG, no_surf, 0, 10000)

0
2000
2000
4000
4000
6000
6000
8000
8000
10000


In [142]:
fcnt_rg14nmNR

{'Negative': array([0.18015215, 0.15704099, 0.19241926, 0.21183447, 0.18585745]),
 'Positive': array([0.21360916, 0.21785725, 0.21380011, 0.2331797 , 0.20803554]),
 'Polar': array([0.14801791, 0.143621  , 0.14477657, 0.16754355, 0.21175707]),
 'Hydrophobic': array([0.16214583, 0.17617274, 0.147999  , 0.15857685, 0.12245538]),
 'Aromatic': array([0.29607495, 0.30530802, 0.30100506, 0.22886543, 0.27189455]),
 'total_frac': array([0.23819302, 0.18685832, 0.16632444, 0.1026694 , 0.12731006])}

In [143]:
fc14nm_meanNR = np.array([np.mean(fcnt_rg14nmNR['Negative']), np.mean(fcnt_rg14nmNR['Positive'])
                        ,np.mean(fcnt_rg14nmNR['Polar']),np.mean(fcnt_rg14nmNR['Hydrophobic'])
                        , np.mean(fcnt_rg14nmNR['Aromatic'])])
fc14nm_meanNR

array([0.18546086, 0.21729635, 0.16314322, 0.15346996, 0.2806296 ])

In [144]:
fc14nm_stdNR = np.array([np.std(fcnt_rg14nmNR['Negative']), np.std(fcnt_rg14nmNR['Positive'])
                       ,np.std(fcnt_rg14nmNR['Polar']),np.std(fcnt_rg14nmNR['Hydrophobic'])
                       , np.std(fcnt_rg14nmNR['Aromatic'])])
fc14nm_stdNR

array([0.01777648, 0.00853437, 0.02580657, 0.01793657, 0.02834986])

In [145]:
x_pos = np.arange(5)
width = 0.28
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(x_pos, fc09nm_meanNR, width, yerr=fc09nm_stdNR, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, fc11nm_meanNR, width, yerr=fc11nm_stdNR, ecolor='black',capsize=5, color='c')
plt.bar(x_pos+(2*width), fc14nm_meanNR, width, yerr=fc14nm_stdNR, ecolor='black',capsize=5, color='lightslategray')
plt.title(r'Fractional Contacts Rg Unrestrained', fontsize=15)
plt.xticks(x_pos+width, labels=aa_types, fontsize=12)
plt.ylim(0,0.4)
plt.legend(['Rg = 0.9 nm', 'Rg = 1.1 nm', 'Rg = 1.4 nm'], frameon=False)
plt.ylabel(r'Fractional Contacts', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Fractional Contacts')

### Total fraction of contacts: averages and std dev calc from 5 20 ns blocks

In [146]:
np.mean(fcnt_rg14nmNR['total_frac'])

0.16427104722792607

In [147]:
np.std(fcnt_rg14nmNR['total_frac'])

0.047201135156286184

In [ ]:
prgrp_14nmNR

In [ ]:
# matrix containing the avg number of PLGA residues for each block for each amino acid 
np.where(aa_matx_2nm[0] != 0)

### Avg no. PLGA residues per BSA AA residue group 

In [ ]:
prgrp_2nm

In [ ]:
mean_2nm = np.zeros(shape=5)
std_2nm = np.zeros(shape=5)
count = 0
for key, value in prgrp_2nm.items():
    mpl_2nm = []
    var_2nm = []
    for i in prgrp_2nm[str(key)].flat:
        mpl_2nm.append(i[0])
        var_2nm.append((i[1])**2)
    
    # calc frac cont averages
    mean_2nm[count] = np.mean(mpl_2nm)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_2nm[count] = np.std(mpl_2nm)
    #std_2nm[count] = np.sqrt(np.sum(var_2nm)/5)
    
    count += 1


In [ ]:
mean_2nm

In [ ]:
std_2nm

In [ ]:
mean_15nm

In [ ]:
std_15nm

In [ ]:
x_pos = np.arange(5)
width = 0.28
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, mean_12nm, width, yerr=std_12nm, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, mean_15nm, width, yerr=std_15nm, ecolor='black',capsize=5, color='c')
plt.bar(x_pos+(2*width), mean_2nm, width, yerr=std_2nm, ecolor='black',capsize=5, color='lightslategray')
plt.title(r'No. of PLGA residues Rg restrained', fontsize=15)
plt.xticks(x_pos+width, labels=aa_types, fontsize=12)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm', 'Rg = 2 nm'], frameon=False)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

### Protein/polymer contact map movie

In [ ]:
np.where(trj_load2nm_res[9920] == 0 )

In [ ]:
trj_load2nm_res[9920]

In [ ]:
trj_load2nm_res.shape

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 2 nm restrained", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.94, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_2nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1.03, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('2nm_res.mp4', writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()